# Unificação de pesos do adaptador com os do modelo

### Imports

In [ ]:
from os import makedirs
from os.path import join
from json import load

from unsloth import FastVisionModel
from getpass import getpass

import torch

from scripts.authentication import authenticate_huggingface

import scripts.definitions as defs

### Autenticação

In [ ]:
authenticate_huggingface()

### Configuração

In [ ]:
MODEL = defs.BASE_MODEL_NAME
TEST_ON_TEST = True  # Testa o modelo sobre os dados de teste
TEST_ON_TRAINING = False  # Testa o modelo sobre os dados de treinamento
SAVE_FREQUENCY = 10  # Salva os resultados a cada N iterações
TEMPERATURE = 0.005

with open(join(defs.TRAINING_PATH, 'models.json'), 'r', encoding='utf-8') as file:
    models = {model_name: defs.Model(**model) for model_name, model in load(file).items()}

model_stats = models[MODEL]
model_path = ''

if model_stats.local:
    model_path = join(defs.RESULTS_PATH, 'adapter_weights', MODEL)
else:
    model_path = MODEL

quantized = model_stats.quantized

### Carregamento do modelo

In [ ]:
model, tokenizer = FastVisionModel.from_pretrained(
    model_path,
    load_in_4bit=quantized,  # type: ignore
    use_gradient_checkpointing='unsloth',
)

### Salvamento da versão unificada do modelo

In [ ]:
weights_path = join(defs.RESULTS_PATH, 'weights')

makedirs(weights_path, exist_ok=True)

model.save_pretrained_merged(weights_path, tokenizer, save_method='merged_16bit')

### Envio do modelo para o Hugging Face

In [ ]:
hf_token = getpass('Enter your Hugging Face token: ')

model.push_to_hub_merged('ErFer7/LLaDerm', tokenizer, save_method='merged_16bit', token=hf_token)